In [ ]:
# Импорт необходимых модулей
from analyzers.audio.audio_analyzer import AudioAnalyzer
from tracking.face_reid import FaceReId
from utils.scene_tools import (
   enrich_scenes_with_characters,
   enrich_scenes_with_audio,
   clean_and_merge_short_scenes,
   print_scenes_formatted, cluster_scenes_with_time_windows, resolve_time_overlaps,
)
from utils.audio_tools import extract_audio_ffmpeg
from sentence_transformers import SentenceTransformer

from utils.video_tools import save_scenes_ffmpeg
from analyzers.video.video_analyzer import VideoAnalyzer
from analyzers.music.music_analyzer import MusicAnalyzer
import os
from video.render_video import render_video_with_faces

d:\All_projects\TGU_projects\Hackathon\Kion_practice\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
VIDEO_PATH = "videos/Video_01.mp4"          # Путь к видеофайлу
AUDIO_PATH = "temp/temp_audio.wav"           # Временный WAV
OUTPUT_DIR = "splitted_scenes/"         # Папка для результатов
MIN_SCENE_LENGTH = 2.0                  # Мин. длительность сцены
MAX_SCENE_LENGTH = 300.0                # Макс. допустимая длина сцены (5 мин)

In [ ]:
os.makedirs(OUTPUT_DIR, exist_ok=True)

splitter = MusicAnalyzer(
    video_path=VIDEO_PATH,
    output_dir=OUTPUT_DIR
)
splitter.run(sensitivity=0.88, min_scene_duration=2.0)

# Анализ видео и выделение сцен

### В этой ячейке мы создаём объект VideoAnalyzer и анализируем видео для получения сцены и лиц, привязанных к трекам.


In [ ]:
video_path = VIDEO_PATH

video_analyzer = VideoAnalyzer()

# Анализируем видео — получаем сцены и лица (срезы лиц из кадров)
scenes, track_faces, tracking_frames = video_analyzer.analyze_video(video_path)

Downloading...
From: https://drive.google.com/uc?id=1-CaioD9NaqbHK_kzSMW8VE4_3KcsRjEo
To: C:\Users\Dell\.cache\torch\checkpoints\osnet_ain_x1_0_imagenet.pth
100%|██████████| 10.9M/10.9M [00:01<00:00, 6.87MB/s]


Successfully loaded imagenet pretrained weights from "C:\Users\Dell/.cache\torch\checkpoints\osnet_ain_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
Model: osnet_ain_x1_0
- params: 2,193,616
- flops: 978,878,352


# Идентификация персонажей (Face Re-Identification)

### Здесь используем FaceReId для определения, каким персонажам принадлежат найденные лица.


In [ ]:
face_reid = FaceReId()

# Получаем отображение track_id → персонаж (имя или ID)
track_id_to_person = face_reid.analyze_persons(track_faces)


Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
find model: /home/artniz/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'],

Analyzing faces: 100%|██████████| 222/222 [04:18<00:00,  1.17s/it]


In [ ]:
render_video_with_faces(video_path, "output_with_faces.mp4", tracking_frames, track_id_to_person)

# Обогащение сцен информацией о персонажах

### Связываем сцены с именами персонажей на основе сопоставления track_id → персонаж.


In [ ]:
scene_data = enrich_scenes_with_characters(scenes, track_id_to_person)


# Извлечение аудио из видео

### С помощью ffmpeg извлекаем аудио в формате WAV для последующего анализа.


In [ ]:
audio_path = extract_audio_ffmpeg(video_path)


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

# Анализ аудио: распознавание речи и активность

### Используем AudioAnalyzer для транскрибирования речи и измерения активности звука (энергии).

In [ ]:
audio_analyzer = AudioAnalyzer(audio_path, model_size='small')

# Получаем список сегментов с распознанным теAudioAnalyzerкстом
segments = audio_analyzer.transcribe_audio()

# Получаем значения энергии аудио для каждого временного фрейма
energy = audio_analyzer.detect_audio_activity(frame_duration=1.0)

# Обогащение сцен транскриптами и аудио энергией

### Добавляем к сценам распознанный текст и усреднённые показатели звуковой активности.


In [ ]:
scene_data_copy = scene_data.copy()

scenes_enrich = enrich_scenes_with_audio(scene_data_copy, segments, energy)


# Семантическая группировка и очистка сцен

### Группируем сцены по сходству персонажей, текстов и аудио, затем объединяем слишком короткие сцены.


In [ ]:
scenes_enrich_copy = scenes_enrich.copy()

# Загружаем модель для вычисления семантических эмбеддингов текста
sentenceTransformer = SentenceTransformer('all-mpnet-base-v2', device="cuda")

# Группируем похожие сцены
scenes_grouped = cluster_scenes_with_time_windows(scenes_enrich_copy, sentenceTransformer)

grouped_chapters_no_overlap = resolve_time_overlaps(scenes_grouped)

# Убираем слишком короткие сцены, объединяя их с соседними
scenes_cleaned = clean_and_merge_short_scenes(grouped_chapters_no_overlap, min_duration=2.0, min_words=3)

# Вывод результата

### Печатаем красиво отформатированный отчёт по сценам.


In [ ]:
print_scenes_formatted(scenes_cleaned)



🎬 Сцена 1
⏱ Время: 0.00 – 62.56 сек
🧍 Персонажи (characters): person_0
🔊 Средняя громкость: 0.0984
🗣 Реплика:
"Итак, если ней, и каждый из них под наблюдением, потом никогда не пойдет через обе. через обе. через обе. обе. И если не под наблюдением, то пройдет. Но если потом не пойдет сквозь обе щели. потом не пойдет сквозь обе щели. щели. Согласен. А в чем прикол? Да просто хорошая идея для принтонов футболки. идея для принтонов футболки. Прошу прощения. Секунду. Так, один коризонтальный. Это ИГИСКИЕ 8 по вертикали. На боков 26 по горизонтали ПВС. 14 по вертикали. Палец. Уберите здесь. Тип. 14 по горизонтали Порта Принц. Принц. Столица Гаити, ли. Гаити. Я могу помочь? Да. Да. Скажите, это здесь банкспермы для высокоинтеллектуалов? Если вы спрашиваете, вам не сюда тогда?"
------------------------------------------------------------

🎬 Сцена 2
⏱ Время: 62.56 – 145.14 сек
🧍 Персонажи (characters): person_0
🔊 Средняя громкость: 0.0390
🗣 Реплика:
"Не, я думаю, мы на месте, да. Заполните во

# Сохранение выделенных сцен в отдельные видеофайлы

### Используем ffmpeg для вырезки и сохранения сцен в отдельные файлы.


In [ ]:
output_dir = "output_scenes"

save_scenes_ffmpeg(video_path, scenes_cleaned, output_dir)

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab